In [ ]:
# Defaults
n = 512
cores = 256

In [ ]:
import numpy as np
import pandas as pd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster as Cluster

from SALib.sample import latin
from SALib.analyze import delta as delta_mim
from joblib import Parallel, delayed, parallel_backend

from mhpc_project.matsch_b2 import Variables, CalibrationModel
from geotopy.optim import Loss
from geotopy.measures import KGE
from geotopy.utils import date_parser

In [ ]:
cluster = Cluster()
cluster.scale(cores=cores)
client = Client(cluster)

In [ ]:
observations = pd.read_csv('../data/Matsch B2/obs.csv',
                           na_values=['-9999', '-99.99'],
                           usecols=[0, 7],
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0,
                           squeeze=True)
observations.index.rename('datetime', inplace=True)
model = CalibrationModel('../data/Matsch B2/geotop', run_args={'timeout': 120})
variables = Variables('../data/Matsch B2/variables.csv')
measure = KGE(observations)
loss = Loss(model, variables, measure)

In [ ]:
problem = {'num_vars': variables.num_vars,
           'names': variables.names,
           'bounds': variables.bounds}

samples = latin.sample(problem, n)

with parallel_backend('dask', client=client):
    losses = Parallel(verbose=10)(delayed(loss)(x) for x in samples)
losses = np.array(losses)

indices = np.isfinite(losses)
good_samples = samples[indices]
good_losses = losses[indices]

SA = delta_mim.analyze(problem, good_samples, good_losses)

In [ ]:
SA.sort_values('S1', key=np.abs, ascending=False)